In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-25m.zip

Continuing in background, pid 81400.
Output will be written to ‘wget-log.5’.


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-100k.zip

Continuing in background, pid 81402.
Output will be written to ‘wget-log.6’.


In [ ]:
#!gdown https://drive.google.com/uc?id=1iyh-mmcyBaMxZAk_Ey4ncW0mSPNKYcCn

### Setup

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!unzip -o "ml-1m.zip"  -d  "/content"

Archive:  ml-1m.zip
  inflating: /content/ml-1m/movies.dat  
  inflating: /content/ml-1m/ratings.dat  
  inflating: /content/ml-1m/README   
  inflating: /content/ml-1m/users.dat  


In [ ]:
data = pd.read_csv('ml-1m/ratings.dat', sep='::', header=0, skipinitialspace=True)
data.dropna(inplace=True)


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
movie_10k_df = data

In [ ]:
movie_10k_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [ ]:

# y = movie_10k_df.groupby('userID').count()['rating']>=200
# ids = y[y].index
# movie_10k_df = movie_10k_df[movie_10k_df['userID'].isin(ids)]

# y = movie_10k_df.groupby('itemID').count()['rating']>=100
# ids = y[y].index
# movie_10k_df = movie_10k_df[movie_10k_df['itemID'].isin(ids)]

In [ ]:
movie_10k_df['userID']=movie_10k_df['userID'].astype(str)
movie_10k_df['itemID']=movie_10k_df['itemID'].astype(str)

In [ ]:
len(list(movie_10k_df['userID'].unique()))

6040

In [ ]:
len(list(movie_10k_df['itemID'].unique()))

3706

In [ ]:
movie_10k_df.shape

(1000208, 4)

In [ ]:
# movie_10k_df.rename(columns = {'userId':'userID'}, inplace = True)
# movie_10k_df.rename(columns = {'movieId':'itemID'}, inplace = True)

# Details about dataset

In [ ]:
summary_df=pd.DataFrame(columns=['Dataset','Algorithm','Sparsity','Precision@k'])

In [ ]:
userID="userID"
itemID="itemID"
score="rating"
timestamp='time'

# Sparsity 

In [ ]:
from scipy.sparse import csr_matrix
def checkSparsity(main_df):
   
    # pivot_df = main_df.pivot_table(
    #     index='itemID',
    #     columns='userID',
    #     values='rating'
    # )
    totalUsers=main_df.userID.unique().shape[0]
    totalitems= main_df.itemID.unique().shape[0]
    total=totalUsers*totalitems
    totalreconds=main_df.shape[0]
    sparsity=(total-totalreconds)/total
    return sparsity

In [ ]:

sparsity_movielens=checkSparsity(movie_10k_df)
sparsity_movielens

0.9553164190519758

In [ ]:
final_movie_users=len(movie_10k_df.userID.unique())
final_movie_items=len(movie_10k_df.itemID.unique())
final_movie_rows=movie_10k_df.shape[0]

In [ ]:
df2 = {'Dataset':"MovieLens",\
       'Sparsity':sparsity_movielens,\
       'Total Users':final_movie_users,\
       'Total Items':final_movie_items,\
       'Total Rows':final_movie_rows}
print(df2)

{'Dataset': 'MovieLens', 'Sparsity': 0.9553164190519758, 'Total Users': 6040, 'Total Items': 3706, 'Total Rows': 1000208}


<!-- Observations :
1. Very sparse dataset sparsity 0.98
2. Many items and users in the dataset has less interaction data -->

<!-- ### Divide dataset based on following:
- All users rated very few items - will use contant based filtering method.To get threshold value for filtering we first find median of number of rating given by each users.Next choose value less then that median value such that we decrese sparsity as much as possible(less then .95)
- For other data we will go with collaborative filtering or other deep learning methods -->

In [ ]:
# movie_10k_df['rating']=1
# amazon_df['rating']=1
# book_df['rating']=1
# food_df['rating']=1
# ciao_df['rating']=1
# ecommerce_df['rating']=1

# Tensorflow recommenders

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs
from typing import Dict, Text

In [ ]:

movie_10k_df["timestamp"] = movie_10k_df["timestamp"].astype(int)

In [ ]:
rating_dict = movie_10k_df[['userID', 
                                      'itemID',
                                      'timestamp'
                                      ,'rating'
                                      ]]
items_dict = movie_10k_df[['itemID']].drop_duplicates()


In [ ]:
#rating_dict = {name: np.array(value) for name, value in rating_dict.items()}
ratings = tf.data.Dataset.from_tensor_slices(dict(rating_dict))

## item features

#items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(dict(items_dict))

In [ ]:
items

<TensorSliceDataset element_spec={'itemID': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
for x in items.take(2).as_numpy_iterator():
  print(x)

{'itemID': b'661'}
{'itemID': b'914'}


In [ ]:
for x in ratings.take(2).as_numpy_iterator():
  print(x)

{'userID': b'1', 'itemID': b'661', 'timestamp': 978302109, 'rating': 3}
{'userID': b'1', 'itemID': b'914', 'timestamp': 978301968, 'rating': 3}


In [ ]:

# Select the basic features.
ratings = ratings.map(lambda x: {
    'userID' : x['userID'], 
    'itemID' : x['itemID'], 
    'timestamp' : x['timestamp'], 
    'rating' : x['rating'],
    
})

items = items.map(lambda x: x['itemID'])

In [ ]:
ratings

<MapDataset element_spec={'userID': TensorSpec(shape=(), dtype=tf.string, name=None), 'itemID': TensorSpec(shape=(), dtype=tf.string, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [ ]:
# userIds    = movie_10k_df.userID.unique()
# productIds = movie_10k_df.itemID.unique()
# total_ratings= len(movie_10k_df.index)

In [ ]:

user_ids =ratings.batch(1_000_000).map(lambda x: x["userID"])
items_list =items.batch(1_000)

In [ ]:
unique_items = np.unique(np.concatenate(list(items_list),axis =0))


In [ ]:
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
unique_user_ids[:10]


array([b'1', b'10', b'100', b'1000', b'1001', b'1002', b'1003', b'1004',
       b'1005', b'1006'], dtype=object)

In [ ]:
total=ratings.__len__().numpy()
train_size=(int) (total * .8 )
test_size=(int) (total * .2 )

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(total, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(train_size)
test = shuffled.skip(train_size).take(test_size)

In [ ]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)


#Tensorflow retrival and Ranking

In [ ]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32
        max_tokens = 10_000

        ## user id
        self.user_embedding = tf.keras.Sequential([
                                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                                    vocabulary=unique_user_ids, mask_token=None),
                                                    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
                                                    ])
       
      
        self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
        ])
        self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
        )

        self.normalized_timestamp.adapt(timestamps)

        
        # self.catagory_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #                                                                                 max_tokens=max_tokens)
        # self.catagory_text_embedding = tf.keras.Sequential([
        #                           self.catagory_vectorizer,
        #                           tf.keras.layers.Embedding(max_tokens, self.embedding_dimension, mask_zero=True),
        #                           tf.keras.layers.GlobalAveragePooling1D(),
        #                         ])

        # self.catagory_vectorizer.adapt(category)
              
        # zip
        # self.zip_embedding = tf.keras.Sequential([
        #                           tf.keras.layers.experimental.preprocessing.StringLookup(
        #                             vocabulary=unique_zip, mask_token=None),
        #                           tf.keras.layers.Embedding(len(unique_zip) + 1, self.embedding_dimension)
        #                         ])
        
        # self.zip_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #                                                                                 max_tokens=max_tokens)
        # self.zip_text_embedding = tf.keras.Sequential([
        #                           self.zip_vectorizer,
        #                           tf.keras.layers.Embedding(max_tokens, self.embedding_dimension, mask_zero=True),
        #                           tf.keras.layers.GlobalAveragePooling1D(),
        #                         ])

        # self.zip_vectorizer.adapt(zip)
        

    def call(self, inputs):
        # Take the input dictionary, pass it through each input layer,
        # and concatenate the result.
        return tf.concat([
            self.user_embedding(inputs["userID"]),
            self.timestamp_embedding(inputs["timestamp"]),
            tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
        ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding user queries
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.embedding_model = UserModel()
            
        self.dense_layers = tf.keras.Sequential()
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

      
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32

        #max_tokens = 10_000

        self.item_embedding = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=unique_items,mask_token=None),
          tf.keras.layers.Embedding(len(unique_items) + 1, self.embedding_dimension)
        ])

        # self.item_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #     max_tokens=max_tokens)

        # self.item_vectorizer.adapt(items)
      
    def call(self, items):
        return tf.concat([
            self.item_embedding(items)
         
        ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):
    """Model for encoding items."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding items.

        
        """
        super().__init__()

        self.embedding_model = ItemModel()

         # Then construct the layers.
        self.dense_layers = tf.keras.Sequential()

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class FinalModel(tfrs.models.Model):

    def __init__(self, layer_sizes, projection_dim=None ):
        super().__init__()
        
        self.query_model : tf.keras.Model = QueryModel(layer_sizes)
        self.candidate_model : tf.keras.Model = CandidateModel(layer_sizes)
        
        ## rating and retrieval task.
        self.layer_sizes=layer_sizes
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
                 
        self.retrieval_task : tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items.batch(128).map(self.candidate_model)
            )
        )

       

    def compute_loss(self, features, training=False):
        
      
        ratings = features.pop("rating")
        
        query_embeddings = self.query_model({
            "userID": features["userID"],
            "timestamp": features["timestamp"]
        })
    
        item_embeddings = self.candidate_model(features["itemID"])       
        retrieval_loss = self.retrieval_task(query_embeddings, item_embeddings)
    
    
        return self.retrieval_task(query_embeddings, item_embeddings)

In [ ]:
cached_train = train.shuffle(train_size).batch(26000).cache()

cached_test = test.batch(5024).cache()

In [ ]:


model = FinalModel([128,32],
                      projection_dim=None)



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
import time
now = time.time()
model.fit(cached_train,epochs=1)
later = time.time()
difference = int(later - now)


31/31 [==============================] - 217s 7s/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0020 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0051 - factorized_top_k/top_50_categorical_accuracy: 0.0134 - factorized_top_k/top_100_categorical_accuracy: 0.0239 - loss: 285096.2529 - regularization_loss: 0.0000e+00 - total_loss: 285096.2529


In [ ]:
print('total time :'+str(difference))

total time :217


In [ ]:

metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.5f}.")
print(f"Retrieval top-50 accuracy: {metrics['factorized_top_k/top_50_categorical_accuracy']:.5f}.")
print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.5f}.")
print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.5f}.")
print(f"Retrieval top-1 accuracy: {metrics['factorized_top_k/top_1_categorical_accuracy']:.5f}.")

40/40 [==============================] - 88s 1s/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.9996e-05 - factorized_top_k/top_5_categorical_accuracy: 2.5995e-04 - factorized_top_k/top_10_categorical_accuracy: 7.1985e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0082 - factorized_top_k/top_100_categorical_accuracy: 0.0214 - loss: 42360.8373 - regularization_loss: 0.0000e+00 - total_loss: 42360.8373
Retrieval top-100 accuracy: 0.02143.
Retrieval top-50 accuracy: 0.00823.
Retrieval top-10 accuracy: 0.00072.
Retrieval top-5 accuracy: 0.00026.
Retrieval top-1 accuracy: 0.00002.


### Prediction

In [ ]:

# index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
# index.index_from_dataset(
#   tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.candidate_model)))
# )
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.candidate_model)))
)
 
 
_, titles = index({
    "userID": np.array(['25']),
    "timestamp": np.array([879024327])},
    k=10
)
titles[0]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'2168', b'3513', b'3617', b'2916', b'1275', b'1288', b'3756',
       b'3597', b'1834', b'2123'], dtype=object)>

# Deep cross

In [ ]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32
        max_tokens = 10_000

        ## user id
        self.user_embedding = tf.keras.Sequential([
                                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                                    vocabulary=unique_user_ids, mask_token=None),
                                                    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
                                                    ])
       
      
        self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
        ])
        self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
        )

        self.normalized_timestamp.adapt(timestamps)

       
    def call(self, inputs):
        # Take the input dictionary, pass it through each input layer,
        # and concatenate the result.
        return tf.concat([
            self.user_embedding(inputs["userID"]),
            self.timestamp_embedding(inputs["timestamp"]),
            tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
        ], axis=1)

In [ ]:
class QueryModel(tf.keras.Model):
    """Model for encoding user queries."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding user queries
        """
        super().__init__()

        # We first use the user model for generating embeddings.
        self.embedding_model = UserModel()
            

        # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(tfrs.layers.dcn.Cross(projection_dim=projection_dim,
                                        kernel_initializer="glorot_uniform"))

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        
        self.embedding_dimension = 32

        #max_tokens = 10_000

        self.item_embedding = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=unique_items,mask_token=None),
          tf.keras.layers.Embedding(len(unique_items) + 1, self.embedding_dimension)
        ])

        # self.item_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        #     max_tokens=max_tokens)

        # self.item_vectorizer.adapt(items)
      
    def call(self, items):
        return tf.concat([
            self.item_embedding(items)
         
        ], axis=1)

In [ ]:
class CandidateModel(tf.keras.Model):
    """Model for encoding items."""

    def __init__(self, layer_sizes, projection_dim=None):
        """Model for encoding items.

        
        """
        super().__init__()

        self.embedding_model = ItemModel()

         # Then construct the layers.
        self.dense_layers = tf.keras.Sequential(tfrs.layers.dcn.Cross(projection_dim=projection_dim,
                                                kernel_initializer="glorot_uniform"))

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(tf.keras.layers.Dense(layer_size))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)

In [ ]:
class CrossDNNModel(tfrs.models.Model):

    def __init__(self, layer_sizes,projection_dim=None ):
        super().__init__()
        
        self.query_model : tf.keras.Model = QueryModel(layer_sizes)
        self.candidate_model : tf.keras.Model = CandidateModel(layer_sizes)
        
        ## rating and retrieval task.
        
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
                 
        self.retrieval_task : tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items.batch(128).map(self.candidate_model)
            )
        )


    def compute_loss(self, features, training=False):
        
        
        ratings = features.pop("rating")
        
        query_embeddings = self.query_model({
            "userID": features["userID"],
            "timestamp": features["timestamp"],
        })
    
        item_embeddings = self.candidate_model(features["itemID"])       
        retrieval_loss = self.retrieval_task(query_embeddings, item_embeddings)
    
    
        return self.retrieval_task(query_embeddings, item_embeddings)

In [ ]:


model = CrossDNNModel([128,32],
                      projection_dim=None)



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
import time
now = time.time()
model.fit(cached_train,epochs=1)
later = time.time()
difference = int(later - now)


31/31 [==============================] - 201s 6s/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0117 - factorized_top_k/top_5_categorical_accuracy: 0.0172 - factorized_top_k/top_10_categorical_accuracy: 0.0213 - factorized_top_k/top_50_categorical_accuracy: 0.0459 - factorized_top_k/top_100_categorical_accuracy: 0.0716 - loss: 330097.3633 - regularization_loss: 0.0000e+00 - total_loss: 330097.3633


In [ ]:
print('total time :'+str(difference))

total time :201


In [ ]:

metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.5f}.")
print(f"Retrieval top-50 accuracy: {metrics['factorized_top_k/top_50_categorical_accuracy']:.5f}.")
print(f"Retrieval top-10 accuracy: {metrics['factorized_top_k/top_10_categorical_accuracy']:.5f}.")
print(f"Retrieval top-5 accuracy: {metrics['factorized_top_k/top_5_categorical_accuracy']:.5f}.")
print(f"Retrieval top-1 accuracy: {metrics['factorized_top_k/top_1_categorical_accuracy']:.5f}.")

40/40 [==============================] - 55s 1s/step - root_mean_squared_error: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 3.8492e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0029 - factorized_top_k/top_10_categorical_accuracy: 0.0060 - factorized_top_k/top_50_categorical_accuracy: 0.0276 - factorized_top_k/top_100_categorical_accuracy: 0.0490 - loss: 42403.4554 - regularization_loss: 0.0000e+00 - total_loss: 42403.4554
Retrieval top-100 accuracy: 0.04903.
Retrieval top-50 accuracy: 0.02757.
Retrieval top-10 accuracy: 0.00600.
Retrieval top-5 accuracy: 0.00291.
Retrieval top-1 accuracy: 0.00038.


### Prediction

In [ ]:

index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.candidate_model)))
)
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.candidate_model)))
)
 
 
_, titles = index({
    "userID": np.array(['25']),
    "timestamp": np.array([879024327])},
    k=10
)
titles[0]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'3830', b'1842', b'228', b'3905', b'2930', b'1181', b'447',
       b'808', b'131', b'3652'], dtype=object)>